In [32]:

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D,SpatialDropout1D,GRU,LSTM
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM,GRU,SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

import pandas as pd
from termcolor import colored
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences



In [16]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [17]:

# Import twitter data
print(colored("Loading train and test data", "green"))

tr = pd.read_csv("/content/drive/My Drive/big.csv")
te = pd.read_csv("/content/drive/My Drive/twt_#ifc.csv")

tr.full_text=tr.full_text.astype(str)
te.full_text=te.full_text.astype(str)
print(colored("... \nData loaded", "green"))


Loading train and test data
... 
Data loaded


In [18]:

# Tokenization
print(colored("Tokenizing and padding data", "green"))
tokenizer = Tokenizer(num_words = 2000, split = ' ')
tokenizer.fit_on_texts(tr['full_text'].astype(str).values)
train_tweets = tokenizer.texts_to_sequences(tr['full_text'].astype(str).values)
max_len = max([len(i) for i in train_tweets])
train_tweets = pad_sequences(train_tweets, maxlen = max_len)
test_tweets = tokenizer.texts_to_sequences(te['full_text'].astype(str).values)
test_tweets = pad_sequences(test_tweets, maxlen = max_len)
print(colored("Tokenizing and padding complete", "yellow"))

Tokenizing and padding data
Tokenizing and padding complete


In [37]:
# Building the model
print(colored("Creating the GRU + CNN model", "yellow"))
modelgcnn = Sequential()
modelgcnn.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))
modelgcnn.add(SpatialDropout1D(0.4))
modelgcnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(8, 1)))
modelgcnn.add(MaxPooling1D(pool_length=(2)))
modelgcnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(8, 1)))
modelgcnn.add(GRU(128))


modelgcnn.add(Dense(3, activation = 'softmax'))
modelgcnn.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modelgcnn.summary()

Creating the GRU + CNN model


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(8, 1), padding="same")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(8, 1), padding="same")`
  


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 37, 128)           256000    
_________________________________________________________________
spatial_dropout1d_10 (Spatia (None, 37, 128)           0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 37, 64)            24640     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 18, 64)            0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 18, 64)            12352     
_________________________________________________________________
gru_5 (GRU)                  (None, 128)               74112     
_________________________________________________________________
dense_7 (Dense)              (None, 3)               

In [38]:
# Training the model
print(colored("Training the GRU + CNN model", "green"))
history = modelgcnn.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the GRU + CNN model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 17s 831us/step - loss: 0.8062 - accuracy: 0.6071
Epoch 2/3
20741/20741 [==============================] - 17s 809us/step - loss: 0.5847 - accuracy: 0.7496
Epoch 3/3
20741/20741 [==============================] - 17s 805us/step - loss: 0.5219 - accuracy: 0.7851


In [39]:
 print(colored("Testing the GRU + CNN model", "green"))

loss, accuracy = modelgcnn.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))
x1=accuracy
score, accuracy = modelgcnn.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))

y1=accuracy

Testing the GRU + CNN model
20741/20741 [==============================] - 4s 188us/step
Train accuracy: 0.8348681330680847
1749/1749 [==============================] - 0s 182us/step
Test accuracy: 0.7930245995521545


In [41]:
# Building the model
print(colored("Creating the LSTM + CNN model", "yellow"))
modelcnnl = Sequential()
modelcnnl.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))
modelcnnl.add(SpatialDropout1D(0.4))
modelcnnl.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(8, 1)))
modelcnnl.add(MaxPooling1D(pool_length=(2)))
modelcnnl.add(LSTM(128))

modelcnnl.add(Dense(3, activation = 'softmax'))
modelcnnl.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modelcnnl.summary()

Creating the LSTM + CNN model


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(8, 1), padding="same")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  import sys


Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 37, 128)           256000    
_________________________________________________________________
spatial_dropout1d_12 (Spatia (None, 37, 128)           0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 37, 64)            24640     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 18, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 387       
Total params: 379,843
Trainable params: 379,843
Non-trainable params: 0
_______________________________________________

In [43]:
# Training the model
print(colored("Training the LSTM + CNN model", "green"))
history = modelcnnl.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the LSTM + CNN model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 17s 812us/step - loss: 0.8106 - accuracy: 0.6069
Epoch 2/3
20741/20741 [==============================] - 16s 765us/step - loss: 0.5911 - accuracy: 0.7463
Epoch 3/3
20741/20741 [==============================] - 16s 769us/step - loss: 0.5291 - accuracy: 0.7775


In [44]:
 print(colored("Testing the LSTM + CNN model", "green"))

loss, accuracy = modelcnnl.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))
x2=accuracy
score, accuracy = modelcnnl.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))
y2=accuracy

Testing the LSTM + CNN model
20741/20741 [==============================] - 4s 203us/step
Train accuracy: 0.8182344436645508
1749/1749 [==============================] - 0s 192us/step
Test accuracy: 0.7901657819747925


In [48]:
# Building the model
print(colored("Creating the RNN + CNN model", "yellow"))
modelrcnn = Sequential()
modelrcnn.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))
modelrcnn.add(SpatialDropout1D(0.4))
modelrcnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(8, 1)))
modelrcnn.add(MaxPooling1D(pool_length=(2)))
modelrcnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(8, 1)))

modelrcnn.add(SimpleRNN(128))

modelrcnn.add(Dense(3, activation = 'softmax'))
modelrcnn.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modelrcnn.summary()

Creating the RNN + CNN model
Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 37, 128)           256000    
_________________________________________________________________
spatial_dropout1d_14 (Spatia (None, 37, 128)           0         
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 37, 64)            24640     
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 18, 64)            0         
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 18, 64)            12352     
_________________________________________________________________
simple_rnn_13 (SimpleRNN)    (None, 128)               24704     
_________________________________________________________________
dense_9 (Dense)         

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(8, 1), padding="same")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(8, 1), padding="same")`
  


In [49]:
# Training the model
print(colored("Training the RNN + CNN model", "green"))
history = modelrcnn.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the RNN + CNN model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 9s 435us/step - loss: 0.8238 - accuracy: 0.5956
Epoch 2/3
20741/20741 [==============================] - 9s 421us/step - loss: 0.5974 - accuracy: 0.7405
Epoch 3/3
20741/20741 [==============================] - 9s 419us/step - loss: 0.5029 - accuracy: 0.7939


In [50]:
 print(colored("Testing the RNN + CNN model", "green"))

loss, accuracy = modelrcnn.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))
x=accuracy
score, accuracy = modelrcnn.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))
y=accuracy

Testing the RNN + CNN model
20741/20741 [==============================] - 2s 113us/step
Train accuracy: 0.8630731105804443
1749/1749 [==============================] - 0s 113us/step
Test accuracy: 0.802744448184967


In [51]:
from tabulate import tabulate


In [52]:

print(tabulate([
    ['RNN + CNN',x,y], 
    ['GRU + CNN',x1,y1],
    ['LSTM + CNN',x2,y2]], 
    
    headers=['Model', 'Training Accuracy','Testing Accuracy']))

Model         Training Accuracy    Testing Accuracy
----------  -------------------  ------------------
RNN + CNN              0.863073            0.802744
GRU + CNN              0.834868            0.793025
LSTM + CNN             0.818234            0.790166
